In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import duckdb as dd
import polars as pl
import os
import glob
import shutil
import zipfile
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
import plotly.express as px
import librosa
from IPython.display import Audio
import pickle
from joblib import dump, load
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import sys

In [ ]:
# !pip download scikit-maad==1.4.1 -d /kaggle/working/mysitepackages/sklearn_maad_pkg
#!pip download ffmpeg-python==0.2.0 -d /kaggle/working/mysitepackages/ffmpeg_pkg

In [ ]:
"""from zipfile import ZipFile
import os

dirName = '/kaggle/working/mysitepackages'
zipName = 'ffmpeg_pkg.zip'

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)"""

In [ ]:
#!pip install ffmpeg-python --no-index --find-links=file:///kaggle/input/ffmpeg-pkg/kaggle/working
#!pip install scikit-maad --no-index --find-links=file:///kaggle/input/sklearn-maad-offline/kaggle/working/mysitepackages/sklearn_maad_pkg

In [ ]:
"""from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from maad import sound, rois, features
from maad.util import (
   power2dB, plot2d, format_features, read_audacity_annot,
   overlay_rois, overlay_centroid
   )"""

In [ ]:
# Function to extract features from audio file
"""def extract_features(file_path, offset=0.0, duration=5.0):
    # Load audio file
    audio, sample_rate = librosa.load(path=file_path, sr=32000, offset=offset, duration=duration)
    # Extract features using Mel-Frequency Cepstral Coefficients (MFCC)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    # Flatten the features into a 1D array
    flattened_features = np.mean(mfccs.T, axis=0)
    return flattened_features

def audio_classification(file_path, offset=0.0, duration=5.0):
    extracted_features = extract_features(file_path, offset=offset, duration=duration).reshape(1, -1)
    y_predict = random_forest_model.predict_proba(extracted_features)
    
    return y_predict"""

In [2]:
config = {}
Test = True

if Test:
    config['root_file_path'] = '/kaggle/input/birdclef-2024/unlabeled_soundscapes/'
    config['start'] = 7000
    config['end'] = 8200
else:
    config['root_file_path'] = '/kaggle/input/birdclef-2024/test_soundscapes/'

In [3]:
# test_soundscapes = '/kaggle/input/birdclef-2024/unlabeled_soundscapes'
# test_soundscapes = '/kaggle/input/birdclef-2024/test_soundscapes'

if Test:
    filenames_with_path = glob.glob(f"{config['root_file_path']}*.ogg")
    # filenames = [('soundscape_' + os.path.basename(filename)) for filename in filenames_with_path]
    filenames = [os.path.basename(filename) for filename in filenames_with_path][config['start']:config['end']]

    print(len(filenames))
else:
    filenames_with_path = glob.glob(f"{config['root_file_path']}*.ogg")
    # filenames = [('soundscape_' + os.path.basename(filename)) for filename in filenames_with_path]
    filenames = [os.path.basename(filename) for filename in filenames_with_path]
    
    print(len(filenames))

1200


In [4]:
durations = {}

for file in tqdm(filenames):
    full_path = config['root_file_path']+file
    audio, sample_rate = librosa.load(path=full_path, sr=32000)
    if round(len(audio) / sample_rate) <= 240:
        durations[file] = round(len(audio) / sample_rate) + 5
    else:
        durations[file] = 240+5

100%|██████████| 1200/1200 [06:07<00:00,  3.27it/s]


In [16]:
#import time

"""durations = {}

def get_file_durations(filename):
    # print("Task assigned to thread: {}".format(threading.current_thread().name))
    
    full_path = config['root_file_path']+filename
    audio, sample_rate = librosa.load(path=full_path, sr=32000)
    if round(len(audio) / sample_rate) <= 240:
        durations[filename] = round(len(audio) / sample_rate) + 5
    else:
        durations[filename] = 240+5
            
    # return durations

vect_func = np.vectorize(get_file_durations)

#%timeit 
vect_func(filenames)"""

KeyboardInterrupt: 

In [5]:
len(durations)

1200

In [6]:
chunk_feature_dict = {}

for file in tqdm(filenames):
    full_path = config['root_file_path']+file
    duration = durations[file]
    for j in range(5,duration,5):
        f_offset = float(j-5)
        audio, sample_rate = librosa.load(path=full_path, sr=32000, offset=f_offset, duration=5.0)
        cur_file = file.replace('.ogg','')
        chunk = cur_file+'_'+str(j)
        mfccs = librosa.feature.mfcc(y=audio, sr=32000, n_mfcc=40)
        # Flatten the features into a 1D array
        flattened_features = (np.mean(mfccs.T, axis=0)).reshape(1, -1)
        chunk_feature_dict[chunk] = flattened_features
        # audio_chunk_dict[chunk] = audio
        
# del durations

 67%|██████▋   | 799/1200 [28:54<14:30,  2.17s/it]


KeyboardInterrupt: 

In [15]:
del chunk_feature_dict

In [7]:
full_path = '/kaggle/input/birdclef-2024/unlabeled_soundscapes/1000424265.ogg'
audio, sample_rate = librosa.load(path=full_path, sr=32000)
samples_per_segment = sample_rate * 5
total_samples = len(audio)
print(total_samples)

7680000


In [14]:
audio.shape

(7680000,)

In [17]:
# audio[7520000:7520000 + 160000].shape
# mfccs = librosa.feature.mfcc(y=audio[7520000:7520000 + 160000], sr=32000, n_mfcc=40)
flattened_features = (np.mean(mfccs.T, axis=0)).reshape(1, -1)

In [24]:
chunk_feature_dict = {}

for file in tqdm(filenames):
    full_path = config['root_file_path']+file
    audio, sample_rate = librosa.load(path=full_path, sr=32000)
    cur_file = file.replace('.ogg','')
    samples_per_segment = sample_rate * 5
    if len(audio) > 7680000:
        total_samples = 7680000
    else:
        total_samples = len(audio)
        
    for i in range(0, total_samples+160000, samples_per_segment):
        if i + samples_per_segment <= total_samples:
            segment = audio[i:i + samples_per_segment]
            chunk = cur_file+'_'+str(int((i/32000)+5))
            mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
            flattened_features = (np.mean(mfccs.T, axis=0)).reshape(1, -1)
            chunk_feature_dict[chunk] = flattened_features

100%|██████████| 1200/1200 [32:12<00:00,  1.61s/it]


In [23]:
for i in range(0, total_samples+160000, samples_per_segment):
    if i + samples_per_segment <= total_samples:
        print(i)
        print((i/32000)+5)

0
5.0
160000
10.0
320000
15.0
480000
20.0
640000
25.0
800000
30.0
960000
35.0
1120000
40.0
1280000
45.0
1440000
50.0
1600000
55.0
1760000
60.0
1920000
65.0
2080000
70.0
2240000
75.0
2400000
80.0
2560000
85.0
2720000
90.0
2880000
95.0
3040000
100.0
3200000
105.0
3360000
110.0
3520000
115.0
3680000
120.0
3840000
125.0
4000000
130.0
4160000
135.0
4320000
140.0
4480000
145.0
4640000
150.0
4800000
155.0
4960000
160.0
5120000
165.0
5280000
170.0
5440000
175.0
5600000
180.0
5760000
185.0
5920000
190.0
6080000
195.0
6240000
200.0
6400000
205.0
6560000
210.0
6720000
215.0
6880000
220.0
7040000
225.0
7200000
230.0
7360000
235.0
7520000
240.0


In [ ]:
"""for file in tqdm(filenames):
    audio, sample_rate = librosa.load(path=full_path, sr=32000)

    # Calculate the number of samples per segment
    samples_per_segment = sample_rate * 5

    # Split the audio into segments
    total_samples = len(audio)

    segments = []
    for i in range(0, total_samples, samples_per_segment):
        if i + samples_per_segment <= total_samples:
            segment = audio[i:i + samples_per_segment]
            segments.append(segment)"""

In [ ]:
"""chunk_feature_dict = {}
        
for file_name, audio_chunk in tqdm(audio_chunk_dict.items()):
    mfccs = librosa.feature.mfcc(y=audio_chunk, sr=32000, n_mfcc=40)
    # Flatten the features into a 1D array
    flattened_features = (np.mean(mfccs.T, axis=0)).reshape(1, -1)
    chunk_feature_dict[file_name] = flattened_features
    
del audio_chunk_dict"""

In [25]:
meta_data = pl.read_csv('../input/birdclef-2024/train_metadata.csv', low_memory=True)
bird_cols = list(meta_data['primary_label'].unique().sort())

submit = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
submit['row_id'] = 'samples'

submit

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
1,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
2,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495


In [26]:
random_forest_model = load('/kaggle/input/trained-audio-model-v-01/audio_classifier_model.joblib')

In [ ]:
# this is where the inference loop will go

for file_name, feature_set in tqdm(chunk_feature_dict.items()):
    df = pd.DataFrame(random_forest_model.predict_proba(feature_set), columns=bird_cols)
    df.insert(loc=0, column='row_id', value=file_name)
    submit = pd.concat([submit,df]).reset_index(drop=True)
    
i = submit[(submit.row_id == 'samples')].index
submit = submit.drop(i).reset_index(drop=True)

del chunk_feature_dict

 75%|███████▌  | 42961/57251 [22:40<13:42, 17.37it/s]

In [ ]:
#submit.sum(axis=1, numeric_only=True)

In [ ]:
submit

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
assert set(sample_submission.columns) == set(submit.columns)
#submission_df = submission_df[sample_submission.columns]

In [ ]:
submit.to_csv('submission.csv', index=False)